## Big-contest EDA
#### 8. 타순에 따라 득/실, 승패에 주는 영향력이 차이가 있는가

In [2]:
import pandas as pd

In [39]:
def load_Data(name, path = "https://raw.githubusercontent.com/dojinyou/Bigcontest_2020/master/Data/sheet_season_csv/"):
    years = ['2016','2017','2018','2019','2020']
    PATH = path
    df = pd.DataFrame()
    for year in years:
        if year == '2016':
            df = pd.read_csv(PATH + name +'_'+year+'.csv', encoding='CP949')
        else :
            new_df = pd.read_csv(PATH + name+'_'+year+'.csv', encoding='CP949')
            df = pd.concat([df, new_df])
    return df

In [40]:
personal_hitter = load_Data('personal_hitter')
team_pitcher = load_Data('team_pitcher')

# 팀투수 - 승패 결과 추출

In [41]:
win_lose = team_pitcher[['G_ID', 'T_ID', 'WLS']]
win_lose.head()

,G_ID,T_ID,WLS
0,20160401HHLG0,LG,W
1,20160401HHLG0,HH,L
2,20160401HTNC0,NC,W
3,20160401HTNC0,HT,L
4,20160401KTSK0,SK,L


In [42]:
# 무승부 경기 G_ID 추출
draw_g_id = win_lose[win_lose['WLS']=="D"]['G_ID']
draw_g_id = draw_g_id.drop_duplicates()
draw_g_id = draw_g_id.tolist()
len(draw_g_id)

33

In [43]:
personal_hitter['G_ID'].isin(draw_g_id).value_counts()

False    80181
True       921
Name: G_ID, dtype: int64

In [44]:
win_lose['G_ID'].isin(draw_g_id).value_counts()

False    6334
True       66
Name: G_ID, dtype: int64

In [45]:
# 무승부 경기 삭제
print(f"before delete : {len(win_lose)}, after delete : ",end="")
win_lose = win_lose.loc[~win_lose['G_ID'].isin(draw_g_id)]
print(len(win_lose))

before delete : 6400, after delete : 6334


In [46]:
# 무승부 경기 삭제
print(f"before delete : {len(personal_hitter)}, after delete : ",end="")
personal_hitter = personal_hitter.loc[~personal_hitter['G_ID'].isin(draw_g_id)]
print(len(personal_hitter))

before delete : 81102, after delete : 80181


In [47]:
len(personal_hitter['G_ID'].unique())

3167

In [48]:
len(win_lose['G_ID'].unique())

3167

## 개인타자 - 승패 정보 추가

In [49]:
hitters = pd.merge(personal_hitter, win_lose, on=['G_ID','T_ID'], how = 'left')
hitters.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,P_ID,START_CK,BAT_ORDER_NO,PA,...,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN,WLS
0,20160401HHLG0,20160401,HH,LG,0,T,60404,0,3,1,...,0,0,0,0,0,1,0.0,0,0,L
1,20160401HHLG0,20160401,LG,HH,0,B,61102,1,8,3,...,0,0,2,0,0,0,0.0,0,0,W
2,20160401HHLG0,20160401,LG,HH,0,B,61186,1,7,4,...,0,0,1,0,0,0,1.0,1,1,W
3,20160401HHLG0,20160401,LG,HH,0,B,62164,0,9,1,...,0,0,0,0,0,0,0.0,0,0,W
4,20160401HHLG0,20160401,HH,LG,0,T,62700,1,9,2,...,0,0,0,0,1,0,0.0,0,0,L


In [50]:
hitters.columns

Index(['G_ID', 'GDAY_DS', 'T_ID', 'VS_T_ID', 'HEADER_NO', 'TB_SC', 'P_ID',
       'START_CK', 'BAT_ORDER_NO', 'PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2', 'H3',
       'HR', 'SB', 'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'ERR',
       'LOB', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN', 'WLS'],
      dtype='object')

## 불필요한 정보 삭제
1. 'GDAY_DS' : G_ID에 포함
2. 'VS_T_ID' : G_ID에 포함
3. 'HEADER_NO' : 관련성 없음
4. 'TB_SC' : 관련성 없음
5. 'P_ID' : 개인타자 식별x
6. 'START_CK' : 선발여부와 관련 없이 타순으로 병합하여 처리

In [51]:
hitters = hitters.drop(['GDAY_DS','VS_T_ID', 'HEADER_NO', 'TB_SC', 'P_ID','START_CK'], axis=1)

In [52]:
hitters.head()

,G_ID,T_ID,BAT_ORDER_NO,PA,AB,RBI,RUN,HIT,H2,H3,...,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN,WLS
0,20160401HHLG0,HH,3,1,1,0,0,1,0,0,...,0,0,0,0,0,1,0.0,0,0,L
1,20160401HHLG0,LG,8,3,3,0,0,0,0,0,...,0,0,2,0,0,0,0.0,0,0,W
2,20160401HHLG0,LG,7,4,4,2,2,3,0,0,...,0,0,1,0,0,0,1.0,1,1,W
3,20160401HHLG0,LG,9,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,0,W
4,20160401HHLG0,HH,9,2,2,0,1,1,0,0,...,0,0,0,0,1,0,0.0,0,0,L


## 같은 타순의 기록을 병합
- 득점권타율(P_HRA_RT)의 경우 단순합산할 수 없고 재계산이 불가능<br>
- 추가 자료 수집 중 무의미한 정보라고 판단하여 삭제<br>득점권 타율은 얼마나 의미가 있을까?(http://mn.kbs.co.kr/mobile/news/view.do?ncd=4162862)


In [53]:
hitters = hitters.drop('P_HRA_RT',axis = 1)

In [54]:
hitters.head()

,G_ID,T_ID,BAT_ORDER_NO,PA,AB,RBI,RUN,HIT,H2,H3,...,BB,IB,HP,KK,GD,ERR,LOB,P_AB_CN,P_HIT_CN,WLS
0,20160401HHLG0,HH,3,1,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,L
1,20160401HHLG0,LG,8,3,3,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,W
2,20160401HHLG0,LG,7,4,4,2,2,3,0,0,...,0,0,0,1,0,0,0,1,1,W
3,20160401HHLG0,LG,9,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,W
4,20160401HHLG0,HH,9,2,2,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,L


In [55]:
hitters = hitters.groupby(by=['BAT_ORDER_NO','WLS']).sum()
hitters

PA     AB   RBI   RUN   HIT   H2   H3   HR   SB   CS  \
BAT_ORDER_NO WLS                                                            
1            L    14685  13300   843  1449  3390  501   68  184  340  183   
             W    15693  13688  1942  3081  4417  757  119  285  568  220   
2            L    14314  12882   991  1401  3197  508   66  200  261  135   
             W    15357  13300  2269  3087  4405  780  109  394  432  142   
3            L    13971  12535  1354  1427  3293  582   38  328  177   79   
             W    15005  13025  3028  3085  4475  906   74  655  247   82   
4            L    13632  12052  1547  1320  3090  556    9  416   98   61   
             W    14685  12523  3517  2982  4433  844   50  864  182   77   
5            L    13344  11962  1371  1144  2966  503   29  336  112   67   
             W    14362  12522  2769  2521  4044  777   41  599  193   86   
6            L    13010  11703  1137   967  2791  491   29  266  125   90   
             W    13982  12189  2457  2247  3871  762   55  499  221  119   
7            L    12673  11453   963   895  2622  403   32  194  157   89   
             W    13641  11883  2159  2096  3566  657   57  376  220  136   
8            L    12292  11120   808   825  2433  390   28  170  126   81   
             W    13290  11585  1993  1952  3392  622   54  313  176   95   
9            L    11925  10759   697   855  2330  324   36  107  163   76   
             W    12872  11094  1667  1979  3191  578   59  194  286  109   

                  ...   SF    BB   IB   HP    KK   GD  ERR   LOB  P_AB_CN  \
BAT_ORDER_NO WLS  ...                                                       
1            L    ...   68  1066   13  171  2353  257  176  2663     2273   
             W    ...  136  1482   43  232  2126  203  125  2628     3404   
2            L    ...   72  1036    5  175  2600  292  205  2632     2583   
             W    ...  127  1420   36  234  2171  257  166  2673     3695   
3            L    ...   89  1135   29  182  2530  345  204  2844     2970   
             W    ...  185  1491   68  252  2250  289  111  2919     4189   
4            L    ...  115  1283   62  176  2577  362  176  2814     3072   
             W    ...  194  1687  145  271  2267  249   99  2972     4168   
5            L    ...  100  1071   27  185  2662  327  240  2633     3018   
             W    ...  170  1371   47  239  2338  330  146  2795     4032   
6            L    ...   93  1016   25  131  2643  328  266  2626     2657   
             W    ...  163  1286   41  222  2338  273  181  2821     3688   
7            L    ...   69   897   20  171  2664  319  307  2529     2627   
             W    ...  139  1239   55  202  2424  271  220  2639     3649   
8            L    ...   59   824   17  164  2696  288  338  2270     2570   
             W    ...  132  1027   29  242  2302  249  212  2438     3603   
9            L    ...   46   798    8  142  2635  228  374  2108     2444   
             W    ...  141  1053   11  192  2172  229  249  2315     3382   

                  P_HIT_CN  
BAT_ORDER_NO WLS            
1            L         508  
             W        1184  
2            L         581  
             W        1285  
3            L         708  
             W        1501  
4            L         693  
             W        1579  
5            L         688  
             W        1347  
6            L         589  
             W        1237  
7            L         539  
             W        1186  
8            L         471  
             W        1128  
9            L         442  
             W        1003  

[18 rows x 21 columns]

In [56]:
hitters = hitters.reset_index(['WLS'])
hitters

,WLS,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,...,SF,BB,IB,HP,KK,GD,ERR,LOB,P_AB_CN,P_HIT_CN
BAT_ORDER_NO,,,,,,,,,,,,,,,,,,,,,
1,L,14685,13300,843,1449,3390,501,68,184,340,...,68,1066,13,171,2353,257,176,2663,2273,508
1,W,15693,13688,1942,3081,4417,757,119,285,568,...,136,1482,43,232,2126,203,125,2628,3404,1184
2,L,14314,12882,991,1401,3197,508,66,200,261,...,72,1036,5,175,2600,292,205,2632,2583,581
2,W,15357,13300,2269,3087,4405,780,109,394,432,...,127,1420,36,234,2171,257,166,2673,3695,1285
3,L,13971,12535,1354,1427,3293,582,38,328,177,...,89,1135,29,182,2530,345,204,2844,2970,708
3,W,15005,13025,3028,3085,4475,906,74,655,247,...,185,1491,68,252,2250,289,111,2919,4189,1501
4,L,13632,12052,1547,1320,3090,556,9,416,98,...,115,1283,62,176,2577,362,176,2814,3072,693
4,W,14685,12523,3517,2982,4433,844,50,864,182,...,194,1687,145,271,2267,249,99,2972,4168,1579
5,L,13344,11962,1371,1144,2966,503,29,336,112,...,100,1071,27,185,2662,327,240,2633,3018,688


## 승리팀 스탯과 패배팀 스탯 분리(그래프용)

In [57]:
hitters_win = hitters[hitters['WLS']=='W'].drop('WLS',axis=1)

In [58]:
hitters_win

,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,CS,...,SF,BB,IB,HP,KK,GD,ERR,LOB,P_AB_CN,P_HIT_CN
BAT_ORDER_NO,,,,,,,,,,,,,,,,,,,,,
1,15693,13688,1942,3081,4417,757,119,285,568,220,...,136,1482,43,232,2126,203,125,2628,3404,1184
2,15357,13300,2269,3087,4405,780,109,394,432,142,...,127,1420,36,234,2171,257,166,2673,3695,1285
3,15005,13025,3028,3085,4475,906,74,655,247,82,...,185,1491,68,252,2250,289,111,2919,4189,1501
4,14685,12523,3517,2982,4433,844,50,864,182,77,...,194,1687,145,271,2267,249,99,2972,4168,1579
5,14362,12522,2769,2521,4044,777,41,599,193,86,...,170,1371,47,239,2338,330,146,2795,4032,1347
6,13982,12189,2457,2247,3871,762,55,499,221,119,...,163,1286,41,222,2338,273,181,2821,3688,1237
7,13641,11883,2159,2096,3566,657,57,376,220,136,...,139,1239,55,202,2424,271,220,2639,3649,1186
8,13290,11585,1993,1952,3392,622,54,313,176,95,...,132,1027,29,242,2302,249,212,2438,3603,1128
9,12872,11094,1667,1979,3191,578,59,194,286,109,...,141,1053,11,192,2172,229,249,2315,3382,1003


In [59]:
hitters_lose = hitters[hitters['WLS']=='L'].drop('WLS',axis = 1)

In [60]:
hitters_lose

,PA,AB,RBI,RUN,HIT,H2,H3,HR,SB,CS,...,SF,BB,IB,HP,KK,GD,ERR,LOB,P_AB_CN,P_HIT_CN
BAT_ORDER_NO,,,,,,,,,,,,,,,,,,,,,
1,14685,13300,843,1449,3390,501,68,184,340,183,...,68,1066,13,171,2353,257,176,2663,2273,508
2,14314,12882,991,1401,3197,508,66,200,261,135,...,72,1036,5,175,2600,292,205,2632,2583,581
3,13971,12535,1354,1427,3293,582,38,328,177,79,...,89,1135,29,182,2530,345,204,2844,2970,708
4,13632,12052,1547,1320,3090,556,9,416,98,61,...,115,1283,62,176,2577,362,176,2814,3072,693
5,13344,11962,1371,1144,2966,503,29,336,112,67,...,100,1071,27,185,2662,327,240,2633,3018,688
6,13010,11703,1137,967,2791,491,29,266,125,90,...,93,1016,25,131,2643,328,266,2626,2657,589
7,12673,11453,963,895,2622,403,32,194,157,89,...,69,897,20,171,2664,319,307,2529,2627,539
8,12292,11120,808,825,2433,390,28,170,126,81,...,59,824,17,164,2696,288,338,2270,2570,471
9,11925,10759,697,855,2330,324,36,107,163,76,...,46,798,8,142,2635,228,374,2108,2444,442


## 그래프

In [61]:
import chart_studio.plotly as py
import cufflinks as cf
import plotly.graph_objects as go
cf.go_offline(connected=True)

In [62]:
def make_figure(col_name):
    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            name = 'Winner',
            x=hitters_win.index, 
            y=hitters_win[col_name], 
            text=hitters_win[col_name], 
            textposition='auto', 
            texttemplate='%{text:.4f}'
        )
    )
    fig.add_trace(
        go.Bar(
            name = 'Loser',
            x=hitters_lose.index, 
            y=hitters_lose[col_name], 
            text=hitters_lose[col_name], 
            textposition='auto', 
            texttemplate='%{text:.4f}'
        )
    )
    return fig

## 단순 스탯 합산값 비교 그래프

In [63]:
for col_name in hitters_win.columns:
    fig = make_figure(col_name)
    fig.update_layout(
        {
            "title": {
                "text": "<b>승패팀의 타순별 "+col_name+"스탯그래프</b>",
                "x": 0.5,
                "y": 0.9,
                "font": {
                    "size": 25
                }
            },
            "xaxis": {
                "title": "타순",
                "showticklabels":True,
                "dtick": "M1",
                "tickfont": {
                    "size": 12                
                }
            }
        }
    )
    fig.show()

## 파생 스탯 비교 그래프
- nc_blog - 야구 데이터 분석 #9 야구 지표 계산법 (타격 지표 편)<br>(https://blog.ncsoft.com/%ec%95%bc%ea%b5%ac-%eb%8d%b0%ec%9d%b4%ed%84%b0-%eb%b6%84%ec%84%9d-9-%ec%95%bc%ea%b5%ac-%ec%a7%80%ed%91%9c-%ea%b3%84%ec%82%b0%eb%b2%95-%ed%83%80%ea%b2%a9-%ec%a7%80%ed%91%9c-%ed%8e%b8/)

1. 타율(Batting Average)<br> 계산법 - (HIT+H2+H3+RUN)/AB
2. 출루율(On Base Percentage, On Base Average)<br> 계산법 - (HIT+H2+H3+RUN+BB+HP)/(AB+BB+HP+SF)
3. 장타율(Slugging Percentage)<br> 계산법 - (HIT+H2*2+H3*3+RUN*4)/AB

In [64]:
derived = {
    'BA':(hitters_win['HIT']+hitters_win['H2']+hitters_win['H3']+hitters_win['HR'])/hitters_win['AB'],
    'OBP':(hitters_win['HIT']+hitters_win['H2']+hitters_win['H3']+hitters_win['HR']+hitters_win['BB']+hitters_win['HP'])/(hitters_win['AB']+hitters_win['BB']+hitters_win['HP']+hitters_win['SF']),
    'SP':(hitters_win['HIT']+hitters_win['H2']*2+hitters_win['H3']*3+hitters_win['HR']*4)/hitters_win['AB']
}
derived_win = pd.DataFrame(derived)
derived_win.head(10)

,BA,OBP,SP
BAT_ORDER_NO,,,
1,0.407510,0.469301,0.542665
2,0.427669,0.486838,0.591579
3,0.469098,0.525179,0.700883
4,0.494370,0.555298,0.776731
5,0.436112,0.494406,0.648219
6,0.425548,0.483045,0.619903
7,0.391820,0.452871,0.551628
8,0.378161,0.435084,0.522227
9,0.362538,0.422035,0.477736


In [65]:
derived = {
    'BA':(hitters_lose['HIT']+hitters_lose['H2']+hitters_lose['H3']+hitters_lose['HR'])/hitters_lose['AB'],
    'OBP':(hitters_lose['HIT']+hitters_lose['H2']+hitters_lose['H3']+hitters_lose['HR']+hitters_lose['BB']+hitters_lose['HP'])/(hitters_lose['AB']+hitters_lose['BB']+hitters_lose['HP']+hitters_lose['SF']),
    'SP':(hitters_lose['HIT']+hitters_lose['H2']*2+hitters_lose['H3']*3+hitters_lose['HR']*4)/hitters_lose['AB']
}
derived_lose = pd.DataFrame(derived)
derived_lose.head(10)

,BA,OBP,SP
BAT_ORDER_NO,,,
1,0.311504,0.368367,0.400902
2,0.308260,0.365831,0.404518
3,0.338333,0.398680,0.469326
4,0.337786,0.405842,0.488964
5,0.320515,0.382190,0.451680
6,0.305648,0.364985,0.420747
7,0.283856,0.343050,0.375447
8,0.271673,0.329498,0.357644
9,0.259968,0.318178,0.326610


4. OPS(On-Base Plus Slugging)<br> 계산법 - 출루율+장타율
5. 순장타율(Isolated Power)<br> 계산법 - 장타율-타율

In [66]:
derived_win['OPS']=derived_win['OBP']+derived_win['SP']
derived_win['IP']=derived_win['SP']-derived_win['BA']
derived_win

,BA,OBP,SP,OPS,IP
BAT_ORDER_NO,,,,,
1,0.407510,0.469301,0.542665,1.011966,0.135155
2,0.427669,0.486838,0.591579,1.078417,0.163910
3,0.469098,0.525179,0.700883,1.226062,0.231785
4,0.494370,0.555298,0.776731,1.332029,0.282360
5,0.436112,0.494406,0.648219,1.142626,0.212107
6,0.425548,0.483045,0.619903,1.102948,0.194356
7,0.391820,0.452871,0.551628,1.004499,0.159808
8,0.378161,0.435084,0.522227,0.957311,0.144066
9,0.362538,0.422035,0.477736,0.899771,0.115197


In [67]:
derived_lose['OPS']=derived_lose['OBP']+derived_lose['SP']
derived_lose['IP']=derived_lose['SP']-derived_lose['BA']
derived_lose

,BA,OBP,SP,OPS,IP
BAT_ORDER_NO,,,,,
1,0.311504,0.368367,0.400902,0.769269,0.089398
2,0.308260,0.365831,0.404518,0.770349,0.096258
3,0.338333,0.398680,0.469326,0.868006,0.130993
4,0.337786,0.405842,0.488964,0.894806,0.151178
5,0.320515,0.382190,0.451680,0.833870,0.131165
6,0.305648,0.364985,0.420747,0.785732,0.115099
7,0.283856,0.343050,0.375447,0.718498,0.091592
8,0.271673,0.329498,0.357644,0.687142,0.085971
9,0.259968,0.318178,0.326610,0.644788,0.066642


In [68]:
for col_name in derived_win.columns:
    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            name = 'Winner',
            x=derived_win.index, 
            y=derived_win[col_name], 
            text=derived_win[col_name], 
            textposition='auto', 
            texttemplate='%{text:.4f}'
        )
    )
    fig.add_trace(
        go.Bar(
            name = 'Loser',
            x=derived_lose.index, 
            y=derived_lose[col_name], 
            text=derived_lose[col_name], 
            textposition='auto', 
            texttemplate='%{text:.4f}'
        )
    )
    fig.update_layout(
        {
            "title": {
                "text": "<b>승패팀의 타순별 "+col_name+"스탯그래프</b>",
                "x": 0.5,
                "y": 0.9,
                "font": {
                    "size": 25
                }
            },
            "xaxis": {
                "title": "타순",
                "showticklabels":True,
                "dtick": "M1",
                "tickfont": {
                    "size": 12                
                }
            }
        }
    )
    fig.show()